<a href="https://colab.research.google.com/github/v-artur/Golden_Oreos/blob/main/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#obtaining the data
!gdown https://drive.google.com/uc?id=1vtZchVzl424pSQBXQ8EBxvVOzcEQPKIp

#extracting it
import zipfile
zip_ref = zipfile.ZipFile("/content/features.zip", 'r')
zip_ref.extractall("/content/features")
zip_ref.close()

Downloading...
From: https://drive.google.com/uc?id=1vtZchVzl424pSQBXQ8EBxvVOzcEQPKIp
To: /content/features.zip
100% 2.14G/2.14G [00:12<00:00, 170MB/s]


In [ ]:
#setting the sample 
feat_path = r'/content/features'

In [ ]:
import numpy as np
import os

In [ ]:
train_subs = ['sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09','sub-10']
val_subs = ['sub-01', 'sub-02']
test_subs = ['sub-03', 'sub-04']

In [ ]:
# concatenating and padding the data
whole_data = []
whole_spec = []

for participant in train_subs:
  data = np.load(os.path.join(feat_path,f'{participant}_feat.npy'))
  whole_data.append(data)
  spec = np.load(os.path.join(feat_path,f'{participant}_spec.npy'))
  whole_spec.append(spec)

new_spec = np.array(spec)

max_len = max([dat.shape[1] for dat in whole_data]) # max number of features
all_len = sum([dat.shape[0] for dat in whole_data]) # sum of all rows

new_data = np.zeros((all_len,max_len))

num_rows = 0
for sub in whole_data:
  row_len = sub.shape[1]
  new_data[num_rows:num_rows+sub.shape[0],0:row_len] = sub
  num_rows += sub.shape[0]

In [ ]:
data = new_data[:700] # csak h gyorsabb legyen
spec = new_spec[:700]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data = scaler.fit_transform(data)

from sklearn.decomposition import PCA
pca = PCA(n_components=300)
data = pca.fit_transform(data)

In [ ]:
data = data.reshape((data.shape[0],data.shape[1],1))
spec = spec.reshape((spec.shape[0],spec.shape[1],1))

## Model 1

In [ ]:
from keras.models import Model
from keras.layers import LSTM
from keras import Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
timesteps = data.shape[1]             # dimensionality of the input sequence
features = 1                          # dimensionality of each input representation in the sequence
outputDim = spec.shape[1]             # dimensionality of the LSTM outputs (Hidden & Cell states)

input = Input(shape=(timesteps, features))
output = LSTM(outputDim)(input)
model_LSTM = Model(inputs=input, outputs=output)

early_stopping=EarlyStopping(patience=25, verbose=1, min_delta=1e-5)
checkpointer = ModelCheckpoint(filepath='weights-lstm.hdf5', save_best_only=True, verbose=1)

model_LSTM.compile(loss='mse', optimizer='adam', metrics=['mse'])
model_LSTM.fit(data, spec, epochs=2, validation_split=0.2, batch_size=64, verbose=1, callbacks=[checkpointer, early_stopping])

Epoch 1/2
9/9 [==============================] - ETA: 0s - loss: 34.7237 - mse: 34.7237
Epoch 1: val_loss improved from inf to 21.97810, saving model to weights-lstm.hdf5
9/9 [==============================] - 4s 206ms/step - loss: 34.7237 - mse: 34.7237 - val_loss: 21.9781 - val_mse: 21.9781
Epoch 2/2
9/9 [==============================] - ETA: 0s - loss: 34.6046 - mse: 34.6046
Epoch 2: val_loss improved from 21.97810 to 21.85045, saving model to weights-lstm.hdf5
9/9 [==============================] - 2s 254ms/step - loss: 34.6046 - mse: 34.6046 - val_loss: 21.8505 - val_mse: 21.8505


In [ ]:
x = data[0].reshape((1,300,1))
model_LSTM.predict(x)

1/1 [==============================] - 1s 1s/step


array([[ 0.02184878,  0.00101783,  0.0498811 ,  0.03227128,  0.00900106,
         0.02459884,  0.03467947, -0.03122801,  0.01736614,  0.00867993,
         0.02060732,  0.02335466,  0.03695188,  0.05550836,  0.02819482,
         0.01989756,  0.00413431,  0.0261831 ,  0.06803001,  0.04563647,
        -0.0118434 ,  0.03158363,  0.01438381]], dtype=float32)

## Model 2

https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

latent_dim = 128
start_symbol = 1.

#### ?
encoder_input_data = data
encoder_input_data[0,0,0] = start_symbol
decoder_input_data = spec
decoder_input_data[0,0,0] = start_symbol

start = np.zeros((1,spec.shape[1],1))
decoder_target_data = np.vstack((start, spec[:-1]))
####

encoder_inputs = Input(shape=(data.shape[1], 1))
encoder = LSTM(units=latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(spec.shape[1], 1))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(spec.shape[1], activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=[decoder_outputs])

model.compile(optimizer='adam', loss='mse', metrics=['mse'])

early_stopping = EarlyStopping(patience=20, verbose=1, min_delta=1e-5)
checkpointer = ModelCheckpoint(filepath='weights.hdf5', save_best_only=True, verbose=1)

history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
                    validation_split=0.2, batch_size=64, epochs=1, callbacks=[checkpointer, early_stopping]) # val?

9/9 [==============================] - ETA: 0s - loss: 34.2520 - mse: 34.2520
Epoch 1: val_loss improved from inf to 21.21002, saving model to weights.hdf5
9/9 [==============================] - 15s 748ms/step - loss: 34.2520 - mse: 34.2520 - val_loss: 21.2100 - val_mse: 21.2100


In [ ]:
# Inference:

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
# def decode_sequence2(input_seq):

#   states_value = encoder_model.predict(input_seq)
#   decoded_sentence = []

#   for t in range(23):
#     target_seq = np.zeros((1, 23, 1))
#     target_seq[0, t, 0] = 1.

#     outputs, h, c = decoder_model.predict(
#         [target_seq] + states_value)

#     decoded_sentence.append(np.argmax(outputs[0, -1, :])) # ?

#     states_value = [h, c]

#   return decoded_sentence

In [ ]:
def decode_sequence(input_seq):
  input_seq[0,0,0] = start_symbol
  states_value = encoder_model.predict(input_seq)

  decoded_sentence = []
  target_seq = np.zeros((1, 23, 1))
  target_seq[0, 0, 0] = start_symbol

  for t in range(1,23):
    
    outputs, h, c = decoder_model.predict(
        [target_seq] + states_value)

    target_seq[0, t, 0] = np.argmax(outputs[0, -1, :])

    states_value = [h, c]

  return target_seq

In [ ]:
x = data[50].reshape((1,300,1))
pred = decode_sequence(x)

1/1 [==============================] - 0s 461ms/step


1/1 [==============================] - 0s 26ms/step


In [ ]:
pred

array([[[ 1.],
        [12.],
        [12.],
        [12.],
        [12.],
        [12.],
        [12.],
        [12.],
        [12.],
        [12.],
        [12.],
        [12.],
        [12.],
        [12.],
        [12.],
        [12.],
        [12.],
        [12.],
        [12.],
        [ 9.],
        [ 9.],
        [ 9.],
        [22.]]])

In [ ]:
pred.shape

(1, 23, 1)

In [ ]:
# https://github.com/dyq0811/EEG-Transformer-seq2seq